In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from scipy.spatial.distance import squareform
import scipy.cluster.hierarchy as sch #might be obsolete
from scipy.cluster.hierarchy import dendrogram, linkage
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import confusion_matrix
from function_pontius import *
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
%matplotlib inline

C:\Users\chimp\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [211]:
def find_medoids(df, metric, nr_clusters):
    med_list = []
    for i in range(nr_clusters):
        med_df = find_cluster(df, metric, i) 
        med_map = med_df['Sum'].idxmin(axis=0, skipna=True)
        med_list.append(med_map)
    return med_list
    
def find_cluster(df=dist_kappa, metric='kappa', cluster=0):
    global df_cluster
    metric_list= ['kappa', 'oa', 'shan', 'simp', 'od', 'oad', 'oqd', 'tca4', 'tca14', 'tca22', 'pland4', 'pland14', 'pland22', 'qd22']
    metric_list.remove(metric)
    df_clean = df_cluster[df_cluster[metric] == cluster].drop(axis='columns', labels=metric_list)
    reduc_list = df_clean.index
    df_reduc = df
    df_reduc = df_reduc[df_reduc.index.isin(reduc_list)]
    df_reduc = df_reduc[reduc_list]
    df_reduc['Sum'] = df_reduc.sum(axis='index')
    return df_reduc

In [125]:
### Load Results ###
#Class 4 Industrial
dist_tca4 = pd.read_csv('C:/LUMOS/MCK/Output_DFs/TCA4_df.csv')
dist_tca4.index = map_list
vals_tca4 = pd.read_csv('C:/LUMOS/MCK/Output_DFs/TCA4_values.csv')

dist_pland4 = pd.read_csv('C:/LUMOS/MCK/Output_DFs/PLAND4_df.csv')
dist_pland4.index = map_list
vals_pland4 =  pd.read_csv('C:/LUMOS/MCK/Output_DFs/PLAND4_values.csv')

#Class 14  Potatoes
dist_tca14 = pd.read_csv('C:/LUMOS/MCK/Output_DFs/TCA14_df.csv')
dist_tca14.index = map_list
vals_tca14 = pd.read_csv('C:/LUMOS/MCK/Output_DFs/TCA14_values.csv')

dist_pland14 = pd.read_csv('C:/LUMOS/MCK/Output_DFs/PLAND14_df.csv')
dist_pland14.index = map_list
vals_pland14 =  pd.read_csv('C:/LUMOS/MCK/Output_DFs/PLAND14_values.csv')


#Class 22 Nature
dist_tca22 = pd.read_csv('E:/thesis_data/Output_DFs/Total Class Area[class 22]_df.csv')
dist_tca22.index = map_list
vals_tca22 = pd.read_csv('E:/thesis_data/Output_DFs/Total Class Area[class 22]_values.csv')

dist_pland22 = pd.read_csv('E:/thesis_data/Output_DFs/PLAND[class 22]_df.csv')
dist_pland22.index = map_list
vals_pland22 =  pd.read_csv('E:/thesis_data/Output_DFs/PLAND[class 22]_values.csv')

# OQD_22
qd22_df = pd.read_csv('E:/thesis_data/Output_DFs/quantitydifferencecategorical_22_df.csv')
np.fill_diagonal(qd22_df.values, 0)

"""Non categorical metrics"""
# Shannon
dist_shan = pd.read_csv('E:/thesis_data/Output_DFs/shannon_df.csv')
dist_shan.index = map_list
shan_vals = pd.read_csv('E:/thesis_data/Output_DFs/shannon_values.csv')
# Simpsons
dist_simp = pd.read_csv('E:/thesis_data/Output_DFs/simp_df.csv')
dist_simp.index = map_list
vals_simp = pd.read_csv('E:/thesis_data/Output_DFs/simp_values.csv')
# Kappa
kappa_df = pd.read_csv('E:/thesis_data/Output_DFs/kappa.csv', index_col= 'Unnamed: 0')
dist_kappa = 1 - kappa_df
# OA
oa_df = pd.read_csv('E:/thesis_data/Output_DFs/overallaccuracy_df.csv')
dist_oa = 1 - oa_df
dist_oa.index = map_list
# OAD
oad_df = pd.read_csv('E:/thesis_data/Output_DFs/overallallocationdifference_df.csv')
oad_df.index = map_list
np.fill_diagonal(oad_df.values, 0)
# OD
od_df = pd.read_csv('E:/thesis_data/Output_DFs/overalldifference_df.csv')
od_df.index = map_list
np.fill_diagonal(od_df.values, 0)
# OQD
oqd_df = pd.read_csv('E:/thesis_data/Output_DFs/overallquantitydifference_df.csv')
oqd_df.index = map_list
np.fill_diagonal(oqd_df.values, 0)

In [3]:
"""Cluster with the found number of clusters"""
kappaclusters = AgglomerativeClustering(n_clusters=7, affinity='precomputed', linkage='complete').fit(dist_kappa)
shannonclusters = AgglomerativeClustering(n_clusters=4, affinity='precomputed', linkage='complete').fit(dist_shan)
simpsonclusters = AgglomerativeClustering(n_clusters=5, affinity='precomputed', linkage='complete').fit(dist_simp)
oaclusters = AgglomerativeClustering(n_clusters=7, affinity='precomputed', linkage='complete').fit(dist_oa)
odclusters = AgglomerativeClustering(n_clusters=7, affinity='precomputed', linkage='complete').fit(od_df)
oadclusters = AgglomerativeClustering(n_clusters=6, affinity='precomputed', linkage='complete').fit(oad_df)
oqdclusters = AgglomerativeClustering(n_clusters=4, affinity='precomputed', linkage='complete').fit(oqd_df)
qd22clusters = AgglomerativeClustering(n_clusters=3, affinity='precomputed', linkage='complete').fit(qd22_df)
tcacluster4 = AgglomerativeClustering(n_clusters=3, affinity='precomputed', linkage='complete').fit(dist_tca4)
tcacluster14 = AgglomerativeClustering(n_clusters=5, affinity='precomputed', linkage='complete').fit(dist_tca14)
tcacluster22 = AgglomerativeClustering(n_clusters=3, affinity='precomputed', linkage='complete').fit(dist_tca22)
plandcluster4 = AgglomerativeClustering(n_clusters=3, affinity='precomputed', linkage='complete').fit(dist_pland4)
plandcluster14 = AgglomerativeClustering(n_clusters=5, affinity='precomputed', linkage='complete').fit(dist_pland14)
plandcluster22 = AgglomerativeClustering(n_clusters=3, affinity='precomputed', linkage='complete').fit(dist_pland22)

labels_kappa = kappaclusters.labels_
labels_shan = shannonclusters.labels_
labels_simp = simpsonclusters.labels_
labels_oa = oaclusters.labels_
labels_od = odclusters.labels_
labels_oad = oadclusters.labels_
labels_oqd = oqdclusters.labels_
labels_qd22 = qd22clusters.labels_
labels_tca4 = tcacluster4.labels_
labels_tca14 = tcacluster14.labels_
labels_tca22 = tcacluster22.labels_
labels_pland4 = plandcluster4.labels_
labels_pland14 = plandcluster14.labels_
labels_pland22 = plandcluster22.labels_

In [209]:
map_list = []
for i in range(2000):
    name = 'map' + str(i)
    map_list.append(name)

df_cluster = pd.DataFrame(index = map_list)
df_cluster['kappa'] = labels_kappa
df_cluster['oa'] = labels_oa
df_cluster['shan'] = labels_shan
df_cluster['simp'] = labels_simp
df_cluster['od'] = labels_od
df_cluster['oad'] = labels_oad
df_cluster['oqd'] = labels_oqd
df_cluster['tca4'] = labels_tca4
df_cluster['tca14'] = labels_tca14
df_cluster['tca22'] = labels_tca22
df_cluster['pland4'] = labels_pland4
df_cluster['pland14'] = labels_pland14
df_cluster['pland22'] = labels_pland22
df_cluster['qd22'] = labels_qd22

df_cluster.head(5)

,kappa,oa,shan,simp,od,oad,oqd,tca4,tca14,tca22,pland4,pland14,pland22,qd22
map0,2,2,0,1,1,1,0,2,2,2,2,3,2,2
map1,6,6,3,4,2,5,2,1,1,0,1,4,0,0
map2,1,1,0,0,4,3,1,0,4,0,0,1,0,0
map3,3,6,1,2,3,0,2,1,1,0,1,2,0,0
map4,6,6,2,4,2,1,2,1,1,2,1,4,2,2


In [229]:
meds_kappa = find_medoids(dist_kappa, 'kappa', 7)
meds_oa = find_medoids(dist_oa, 'oa', 7)
meds_shan = find_medoids(dist_shan, 'shan', 4)
meds_simp = find_medoids(dist_simp, 'simp', 5)
meds_od = find_medoids(od_df, 'od', 7)
meds_oad = find_medoids(oad_df, 'oad', 6)
meds_oqd = find_medoids(oqd_df, 'oqd', 4)
meds_qd22 =  find_medoids(qd22_df, 'qd22', 3)
meds_tca4 = find_medoids(dist_tca4, 'tca4', 3)
meds_tca14 = find_medoids(dist_tca14, 'tca14', 5)
meds_tca22 = find_medoids(dist_tca22, 'tca22', 3)
meds_pland4 = find_medoids(dist_pland4, 'pland4', 3)
meds_pland14 = find_medoids(dist_pland14, 'pland14', 5)
meds_pland22 = find_medoids(dist_pland22, 'pland22', 3)

In [230]:
"""Make all medoid map lists length 7 so they can be used to make a df """
list_of_meds = [meds_kappa, meds_oa, meds_shan, meds_simp, meds_od, meds_oad, meds_oqd, meds_qd22, meds_tca4, meds_tca14, meds_tca22, meds_pland4, meds_pland14, meds_pland22]
def list_filler(entry):
    if len(entry) < 7:
        fillers = 7 - len(entry)
        for i in range(fillers):
            entry.append('-')
            
for entry in list_of_meds:
    list_filler(entry)

In [231]:
df_med = pd.DataFrame({'kappa': meds_kappa,
     'oa': meds_oa,
     'shan': meds_shan,
     'simp': meds_simp,
     'od': meds_od,
     'oad': meds_oad,
     'oqd': meds_oqd,    
     'tca4': meds_tca4,
     'pland4': meds_pland4,                       
     'tca14': meds_tca14,
     'pland14': meds_pland14,                       
     'tca22': meds_tca22,  
     'pland22': meds_pland22
    })
df_med

,kappa,oa,shan,simp,od,oad,oqd,tca4,pland4,tca14,pland14,tca22,pland22
0,map1911,map1911,map1457,map1784,map1911,map678,map1472,map332,map1422,map325,map325,map35,map35
1,map541,map541,map61,map65,map359,map1760,map1057,map217,map264,map296,map564,map8,map8
2,map359,map359,map1248,map1448,map1760,map453,map1978,map604,map1877,map246,map843,map0,map0
3,map1148,map10,map1454,map1830,map1148,map1852,map1833,-,-,map825,map246,-,-
4,map695,map695,-,map857,map541,map1918,-,-,-,map1612,map704,-,-
5,map456,map456,-,-,map695,map263,-,-,-,-,-,-,-
6,map1760,map696,-,-,map456,-,-,-,-,-,-,-,-


In [232]:
df_cluster

,kappa,oa,shan,simp,od,oad,oqd,tca4,tca14,tca22,pland4,pland14,pland22,qd22
map0,2,2,0,1,1,1,0,2,2,2,2,3,2,2
map1,6,6,3,4,2,5,2,1,1,0,1,4,0,0
map2,1,1,0,0,4,3,1,0,4,0,0,1,0,0
map3,3,6,1,2,3,0,2,1,1,0,1,2,0,0
map4,6,6,2,4,2,1,2,1,1,2,1,4,2,2
map5,3,3,0,0,3,0,1,0,3,2,0,2,2,2
map6,3,3,1,2,3,0,2,1,1,2,1,4,2,2
map7,4,4,0,0,5,2,1,0,3,0,0,1,0,0
map8,0,0,0,1,0,0,0,0,0,1,0,0,1,1
map9,3,3,1,4,3,0,2,1,1,2,1,2,2,2
